In [48]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [49]:
#데이터 경로
train_dir = '/data/face1/train'
test_dir = '/data/face1/test'

In [50]:
# 데이터 증강 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 정규화
    rotation_range=20,  # 회전
    width_shift_range=0.2,  # 가로 이동
    height_shift_range=0.2,  # 세로 이동
    zoom_range=0.2,  # 확대/축소
    horizontal_flip=True  # 좌우 반전
)

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     horizontal_flip=True  # 간단한 좌우 반전만 적용
# )

In [51]:
test_datagen = ImageDataGenerator(rescale=1./255)  # 테스트 데이터는 증강하지 않음

In [52]:
# 데이터 로드
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),  # FER-2013 이미지 크기
    color_mode='grayscale',  # 흑백 이미지
    batch_size=32,
    class_mode='categorical'  # 다중 클래스 분류
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [55]:
# MobileNetV3 모델 로드 및 설정
base_model = MobileNetV3Small(
    input_shape=(48, 48, 1),  # 흑백 이미지 입력
    include_top=False,
    weights=None  # 사전 학습된 가중치 사용하지 않음
)
base_model.trainable = True

# Sequential 모델 정의
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),  # 노드 수 축소
    Dropout(0.5),  # 드롭아웃 비율 증가
    Dense(7, activation='softmax')  # 클래스 수에 맞게 출력 정의
])

# 모델 요약 출력
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)   │ (None, 2, 2, 576)      │       938,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,013,591 (3.87 MB)

 Trainable params: 1,001,479 (3.82 MB)

 Non-trainable params: 12,112 (47.31 KB)

In [58]:
# # 모델 컴파일
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # 초기 학습률 낮춤
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# 모델 컴파일
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [57]:
# # 콜백 설정
# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=5,  # 검증 손실이 5 에포크 동안 개선되지 않으면 중단
#     restore_best_weights=True
# )
# reduce_lr = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.5,  # 학습률을 50%로 감소
#     patience=3,  # 3 에포크 동안 개선되지 않으면 감소
#     min_lr=1e-6  # 학습률의 최소값
# )

In [59]:
# 모델 학습
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=50,
    callbacks=[early_stopping]
)

Epoch 1/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 67s 58ms/step - accuracy: 0.2333 - loss: 1.8603 - val_accuracy: 0.2471 - val_loss: 1.8209
Epoch 2/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 51s 57ms/step - accuracy: 0.2559 - loss: 1.8024 - val_accuracy: 0.2471 - val_loss: 1.8154
Epoch 3/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 51s 56ms/step - accuracy: 0.2569 - loss: 1.7892 - val_accuracy: 0.2471 - val_loss: 1.8183
Epoch 4/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 51s 57ms/step - accuracy: 0.2587 - loss: 1.7841 - val_accuracy: 0.2471 - val_loss: 1.8344
Epoch 5/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 51s 56ms/step - accuracy: 0.2588 - loss: 1.7801 - val_accuracy: 0.2471 - val_loss: 1.8389
Epoch 6/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 51s 57ms/step - accuracy: 0.2664 - loss: 1.7731 - val_accuracy: 0.2471 - val_loss: 1.8334
Epoch 7/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 51s 57ms/step - accuracy: 0.2735 - loss: 1.7664 - val_accuracy: 0.1335 - val_loss: 4.0511
